In [14]:
# compare the two text files and print out the different line; if two files are identical, print No difference
def compareFile(path1 = "midsummer.txt",path2 = "recon.txt"):
    f1 = open(path1, "r")  
    f2 = open(path2, "r")   
    i = 0
    for line1 in f1:
        i += 1      
        for line2 in f2:
            if line1 != line2:
                print("Differs at line", i, ":")
                # else print that line from both files
                print("\tFile 1:", line1, end='')
                print("\tFile 2:", line2, end='')
            break
    print("The input file and reconstructed file have no difference")
    # closing files
    f1.close()
    f2.close()

In [15]:
def bin2ascii(string):
    message = ""
    while string != "":
        i = chr(int(string[:8], 2))
        message = message + i
        string = string[8:]
    return(message)

In [16]:
bin2ascii("00000111")

'\x07'

'1011'

In [27]:
# huffman code 
import heapq
# binary tree with parent node value always smaller than the child value.
class huffmancoding:
    def __init__(self,original_path,compressed_path,recon_path):
        self.original_path = original_path
        self.compressed_path = compressed_path
        self.recon_path = recon_path
        self.heap = []
        self.codes = {}
        self.reverse_mapping = {}
    
    class HeapNode:
        def __init__(self,char,frequency):
            self.char = char
            self.freq = frequency
            self.left = None # nodes of higher frequency
            self.right = None # nodes of lower frequency
            
        def __lt__(self, other):
            return self.freq < other.freq

        def __eq__(self, other):
            if(other == None):
                return False
            if(not isinstance(other, HeapNode)):
                return False
            return self.freq == other.freq
        
    def count_frequency(self,text):
        frequency = {}
        for char in text:
            if not char in frequency:
                frequency[char] = 1
            else:
                frequency[char] +=1
        return frequency
    
    def make_heap(self, frequency):
        for key in frequency:
            node = self.HeapNode(key, frequency[key])
            heapq.heappush(self.heap, node)
    
    def combine_nodes(self):
        while(len(self.heap)>1):
            node1 = heapq.heappop(self.heap)
            node2 = heapq.heappop(self.heap)
            merged_node = self.HeapNode(None, node1.freq + node2.freq)
            merged_node.left = node1
            merged_node.right = node2
            heapq.heappush(self.heap, merged_node)

    def make_codes_helper(self, root, current_code):
        if(root == None):
            return
        if(root.char != None):
            self.codes[root.char] = current_code
            self.reverse_mapping[current_code] = root.char
            return

        self.make_codes_helper(root.left, current_code + "0")
        self.make_codes_helper(root.right, current_code + "1")


    def make_codes(self):
        root = heapq.heappop(self.heap)
        current_code = ""
        self.make_codes_helper(root, current_code)


    def get_encoded_text(self, text):
        encoded_text = ""
        for char in text:
            encoded_text += self.codes[char]
        return encoded_text


    def pad_encoded_text(self, encoded_text):
        extra_padding = 8 - len(encoded_text) % 8
        for i in range(extra_padding):
            encoded_text += "0"

        padded_info = "{0:08b}".format(extra_padding)
        encoded_text = padded_info + encoded_text
        return encoded_text


    def get_byte_array(self, padded_encoded_text):
        if(len(padded_encoded_text) % 8 != 0):
            print("Encoded text not padded properly")
            exit(0)

        b = bytearray()
        for i in range(0, len(padded_encoded_text), 8):
            byte = padded_encoded_text[i:i+8]
            b.append(int(byte, 2))
        return b
# compression:
    def compress(self):
        with open(self.original_path,'r+') as filein:
            text = filein.read()
            frequency = self.count_frequency(text)
            self.make_heap(frequency)
            self.combine_nodes()
            self.make_codes()
            compressed_text = self.get_encoded_text(text)
            compressed_text = bin2ascii(str(compressed_text)).encode("utf-8")
            #TODO
            with open(self.compressed_path,'wb') as fileout:
                fileout.write(compressed_text)
                print("size of original text is:",len(text),"letters")
                print("size of compressed text is:",len(compressed_text),"bits")
                print("rate of compression is:",len(text)/len(compressed_text))

# reconstruction:
    def decode_text(self, encoded_text):
        current_code = ""
        decoded_text = ""
        for bit in encoded_text:
            current_code += bit
            if(current_code in self.reverse_mapping):
                character = self.reverse_mapping[current_code]
                decoded_text += character
                current_code = ""

        return decoded_text


    def recon(self):
        with open(self.compressed_path,'r+', encoding="utf-8") as filein:
            compressed_text = filein.read()
            compressed_text = compressed_text.rstrip()
            recon_text = self.decode_text(compressed_text)
            with open(self.recon_path,'w') as fileout:
                fileout.write(recon_text)


In [28]:
# Testing:
original_path = 'midsummer.txt'
compressed_path = 'compressed.txt'
recon_path = 'recon.txt'
# Huffman testing:
print("Huffman testing:")
huffman = huffmancoding(original_path, compressed_path, recon_path)
huffman.compress()
huffman.recon()
compareFile(original_path, recon_path)

Huffman testing:
size of original text is: 92603 letters
size of compressed text is: 85283 bits
rate of compression is: 1.085831877396433   8.686655019171464
The input file and reconstructed file have no difference


In [22]:
from bitstring import BitArray, Bits

def lz77(textFile, windowBits, lengthBits):
    with open(textFile, "rb") as inputFile:
        data = BitArray(inputFile.read())
    compressed = compress(data, windowBits, lengthBits)
    decompressed = decompress(compressed, windowBits, lengthBits)
    
    with open("compressed.bin", "wb") as compressedFile:
       compressed.tofile(compressedFile)
    with open("decompressed.txt", "wb") as decompressedFile:
       decompressed.tofile(decompressedFile)
    
    print("Data length: {} bits".format(len(data)))
    print("Compressed length: {} bits".format(len(compressed)))
    if (data == decompressed):
        print("Successful compression and decompression")
    else:
        print("Error in compression and decompression")
    return([len(data),len(compressed)])

def compress(data, windowBits, lengthBits):
    maxWindowLength = 2**windowBits - 1
    bufferLength = 2**lengthBits - 1
    buffer = data[:bufferLength]
    substring = buffer
    compressed = BitArray()
    window = BitArray('')

    #constants in the case that a match is not found
    zeroPos = Bits(uint=0, length=windowBits)
    zeroLength = Bits(uint=0, length=lengthBits)

    bufferPos = 0
    maxLength = len(data)
    while ((bufferPos) < maxLength):
        bufferExtend = min(bufferPos + bufferLength, maxLength)
        buffer = data[bufferPos: bufferExtend]
        bufferStepper = len(buffer) 
        tripletAdded = False
        while bufferStepper > 0 and not tripletAdded:
            substring = buffer[0:bufferStepper]
            
            if (window.find(substring) != ()):
                position =  len(window) - int(window.find(substring)[0])
                
                length = len(substring)
                nextCharIndex = bufferPos + length
                if nextCharIndex > len(data) - 1:
                    nextCharIndex -= 1
                    substring = substring[:-1]
                    length = len(substring)
                nextChar = data[nextCharIndex:nextCharIndex+1]
                
                bitsPosition = Bits(uint=position, length=windowBits)
                bitsLength = Bits(uint=length, length=lengthBits)

                compressedTriplet = bitsPosition + bitsLength + nextChar
                substring += nextChar
                length += 1
                tripletAdded = True
            
            elif len(substring) == 1:
                length = 1
                compressedTriplet = zeroPos + zeroLength + substring
                tripletAdded = True
            bufferStepper -= 1
        bufferPos += length
        window += substring

        if len(window) > maxWindowLength:
            startIndex = len(window) -  maxWindowLength
            window = window[startIndex:]
        compressed += compressedTriplet
        
    return compressed

def decompress(compressed, windowBits, lengthBits):
    decompressedData = BitArray('')
    i = 0
    while i in range(len(compressed)):
        pos = compressed[i:(i+windowBits)].uint
        i += windowBits
        length = compressed[i:(i+lengthBits)].uint
        i += lengthBits
        char = compressed[i:(i+1)]
        i += 1
        if (pos == 0 and length == 0):
            decompressedData += char
        else:
            startPos = len(decompressedData) - pos
            endPos = startPos + length
            substring = decompressedData[startPos:endPos] + char
            decompressedData += substring
    return decompressedData

In [23]:
#Lampel Ziv testing:
print("Lampel Ziv testing:")
[a,b] = lz77(original_path,1,1)

Lempel Ziv testing:
Data length: 740824 bits
Compressed length: 1458126 bits
Successful compression and decompression


In [25]:
print("compression rate: ", a/b, " ", a/b*8)

compression rate:  0.5080658324452071   4.064526659561657


In [27]:
#Lampel Ziv testing:
print("Lampel Ziv testing:")
lampel_ziv77 = [[]]
for i in range (1,11,2):
    for j in range (1,101,10):
        print("Window length: ",j)
        print("L: ",i)
        [a,b] = lz77(original_path,j,i)
        print("compression rate: ", a/b, " ", a/b*8)
        lampel_ziv77.append([j,i,a/b*8])

Lempel Ziv testing:
Window length:  1
L:  1
Data length: 740824 bits
Compressed length: 1458126 bits
Successful compression and decompression
compression rate:  0.5080658324452071   4.064526659561657
Window length:  11
L:  1
Data length: 740824 bits
Compressed length: 4815369 bits
Successful compression and decompression
compression rate:  0.15384573850934372   1.2307659080747497
Window length:  21
L:  1
Data length: 740824 bits
Compressed length: 8519499 bits
Successful compression and decompression
compression rate:  0.0869562869835421   0.6956502958683368
Window length:  31
L:  1
Data length: 740824 bits
Compressed length: 12223629 bits
Successful compression and decompression
compression rate:  0.06060589698852935   0.4848471759082348
Window length:  41
L:  1
Data length: 740824 bits
Compressed length: 15927759 bits
Successful compression and decompression
compression rate:  0.04651150234003415   0.3720920187202732
Window length:  51
L:  1
Data length: 740824 bits
Compressed length

KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
lempel_ziv77 = numpy.array(lempel_ziv77)
print 
plt.scatter(lempel_ziv77[:,0],lempel_ziv77[:,2])
plt.show()